# Topic: Neural Networks !

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

print("imports successful!")

In [ ]:
#perceptron class as one of the building blocks for neural networks

class Perceptron:
    
    def __init__(self, learning_rate=0.1):
        
        self.learning_rate = learning_rate
        self.b = 0.0
        self._w = None
        self._x = np.array([])
        self.misclassified_samples = []
        
    def fit(self, x: np.array, y: np.array, n_iter=10):
        self.b = 0.0
        self._w = np.zeros(x.shape[1])
        self.misclassified_samples = []
        
        for xi, yi in zip(x, y):
            update = self.learning_rate * (yi - self.predict(xi))
            self._b += update
            self._w += update
            errors += int(update != 0.0)
            
        self.misclassified_samples.append(errors)
        
    def f(self, x: np.array) -> float:
        return np.dot(x, self._x) + self._b
    
    def predict(self, x:np.array):
        
        return np.where(self.f(x) >= 0, 1, -1)

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()

x = iris.data
y = iris.target

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5)
classifier = Perceptron(learning_rate=0.01)

#display x_test
print(x_train)

print(y_test)

fig = plt.plot
ax = plt.axes(projection='3d')
ax.set_title('Iris data set')
ax.set_xlabel('Sepal length (cm)')
ax.set_ylabel('Sepal width (cm)')
ax.set_zlabel('Petal Length (cm)')

ax.scatter(x[:50, 0], x[:50, 1], x[:50, 2], color="red")
ax.scatter(x[50:100, 0], x[50:100, 1], x[50:100, 2], color="green")
ax.scatter(x[100:150, 0], x[100:150, 1], x[100:150, 2], color="blue")
plt.show()

In [ ]:
classifier.fit(x_train, y_train)

predictions = classifier.predict(x_test)